# 네이버지도 연동하기

In [1]:
# 1️⃣ geopy 설치
!pip install geopy

In [1]:
# 2️⃣ 모듈 가져오기
from geopy.distance import geodesic

# 3️⃣ 병원 더미 데이터
hospitals = [
    {"name": "OO심리상담센터", "lat": 37.57, "lng": 126.98, "rating": 4.5, "review": "친절하고 좋아요!"},
    {"name": "AA정신건강의학과", "lat": 37.565, "lng": 126.977, "rating": 4.2, "review": "편안했어요!"},
    {"name": "BB마음클리닉", "lat": 37.55, "lng": 126.99, "rating": 4.7, "review": "상담이 좋아요!"}
]

# 4️⃣ 사용자 위치 입력
user_lat = 37.56  # 직접 숫자 넣기
user_lng = 126.97
user_location = (user_lat, user_lng)

# 5️⃣ 거리 계산 + 병원에 추가
for hospital in hospitals:
    hospital_location = (hospital['lat'], hospital['lng'])
    distance_km = geodesic(user_location, hospital_location).km
    hospital['distance_km'] = round(distance_km, 2)

# 6️⃣ 가까운 순 정렬
sorted_hospitals = sorted(hospitals, key=lambda x: x['distance_km'])

# 7️⃣ 출력
for idx, h in enumerate(sorted_hospitals, 1):
    print(f"{idx}. {h['name']}")
    print(f"   거리: {h['distance_km']} km")
    print(f"   평점: {h['rating']}")
    print(f"   리뷰: {h['review']}")
    print("---------------")


1. AA정신건강의학과
   거리: 0.83 km
   평점: 4.2
   리뷰: 편안했어요!
---------------
2. OO심리상담센터
   거리: 1.42 km
   평점: 4.5
   리뷰: 친절하고 좋아요!
---------------
3. BB마음클리닉
   거리: 2.09 km
   평점: 4.7
   리뷰: 상담이 좋아요!
---------------


✔ 1) 좌표 = 정적 데이터
✔ 2) 리뷰/평점 = 외부/사용자/크롤링으로 유동적으로 채움
✔ 3) 없는 건 NULL 처리 + UI에서 비어있다고 표시
✔ 4) API 응답 구조는 항상 동일하게 유지

In [5]:
!pip install fastapi uvicorn

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 18.0 MB/s eta 0:00:00

   ---- -----------------------------------  1/10 [sniffio]
   ------------ ---------------------------  3/10 [h11]
   ------------ ---------------------------  3/10 [h11]
   ------------ ---------------------------  3/10 [h11]
   -------------------- -------------------  5/10 [uvicorn]
   -------------------- -------------------  5/10 [uvicorn]
   -------------------- -------------------  5/10 [uvicorn]
   -------------------- -------------------  5/10 [uvicorn]
   -------------------- -------------------  5/10 [uvicorn]
   -------------------- -------------------  5/10 [uvicorn]
   ------------------------ ---------------  6/10 [pydantic]
   ------------------------ ---------------  6/10 [pydantic]
   ------------------------ ---------------  6/10 [pydantic]
   ------------------------ ---------------  6/10 [pydantic]
   ------------------

In [6]:
from fastapi import FastAPI, Query
from fastapi.responses import JSONResponse
from geopy.distance import geodesic
import json

app = FastAPI()

# JSON 파일에서 병원 데이터 로드
with open('hospitals.json', 'r', encoding='utf-8') as f:
    hospitals = json.load(f)

# 여기서 프론트엔드에서 사용자가 주소입력하면 좌표로 변환, 변환값과 함께 API호출.
@app.get("/api/hospitals")
async def get_hospitals(lat: float = Query(...), lng: float = Query(...)):
    user_location = (lat, lng)

    results = []
    for h in hospitals:
        hospital_location = (h['lat'], h['lng'])
        distance_km = geodesic(user_location, hospital_location).km 
        # geodesic: 두 점사이의 '최단경로거리' 즉, 지구의 곡면을 따라 재는 킬로미터 단위계산
        h_with_distance = h.copy()
        h_with_distance['distance_km'] = round(distance_km, 2)
        results.append(h_with_distance)

    # 거리순 정렬
    results.sort(key=lambda x: x['distance_km'])

    return JSONResponse(content=results)


FileNotFoundError: [Errno 2] No such file or directory: 'hospitals.json'